In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
DIR = '/content/gdrive/My Drive/Flight_delay/'

In [3]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None) 

file_loc = DIR + 'airline_delay.csv'
#print(file_loc)
data = pd.read_csv(file_loc)
print(data.shape)
data.head(5)

(256285, 21)


,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2003,6,AA,American Airlines Inc.,ABQ,"Albuquerque, NM: Albuquerque International Sun...",307.0,56.0,14.68,10.79,19.09,1.48,9.96,1.0,1.0,2530.0,510.0,621.0,676.0,25.0,698.0
1,2003,6,AA,American Airlines Inc.,ANC,"Anchorage, AK: Ted Stevens Anchorage Internati...",90.0,27.0,7.09,2.00,10.75,0.00,7.16,0.0,0.0,1390.0,271.0,83.0,581.0,0.0,455.0
2,2003,6,AA,American Airlines Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",752.0,186.0,33.99,27.82,104.76,1.90,17.53,5.0,0.0,8314.0,1367.0,1722.0,3817.0,139.0,1269.0
3,2003,6,AA,American Airlines Inc.,AUS,"Austin, TX: Austin - Bergstrom International",842.0,174.0,60.24,20.54,47.78,4.69,40.75,9.0,1.0,8344.0,3040.0,1032.0,1835.0,115.0,2322.0
4,2003,6,AA,American Airlines Inc.,BDL,"Hartford, CT: Bradley International",383.0,55.0,14.90,8.91,14.57,0.00,16.61,0.0,0.0,3137.0,815.0,574.0,555.0,0.0,1193.0


In [4]:
data.isnull().sum()

year                     0
month                    0
carrier                  0
carrier_name             0
airport                  0
airport_name             0
arr_flights            362
arr_del15              416
carrier_ct             362
weather_ct             362
nas_ct                 362
security_ct            362
late_aircraft_ct       362
arr_cancelled          362
arr_diverted           362
arr_delay              362
carrier_delay          362
weather_delay          362
nas_delay              362
security_delay         362
late_aircraft_delay    362
dtype: int64

In [0]:
data = data.dropna()

In [6]:
data.isnull().sum()

year                   0
month                  0
carrier                0
carrier_name           0
airport                0
airport_name           0
arr_flights            0
arr_del15              0
carrier_ct             0
weather_ct             0
nas_ct                 0
security_ct            0
late_aircraft_ct       0
arr_cancelled          0
arr_diverted           0
arr_delay              0
carrier_delay          0
weather_delay          0
nas_delay              0
security_delay         0
late_aircraft_delay    0
dtype: int64

In [7]:
# For Machine Learning

data_ML = data[['year','month','carrier','arr_flights','arr_del15']]
data_ML.groupby(['year','month','carrier','arr_flights']).sum()
print(data_ML.shape)

data_ML['delay_rate'] = data_ML.arr_del15/data_ML.arr_flights
data_ML['year'] = (data_ML.year-2003.)/2019
data_ML['month'] = data_ML.month/12


col         = 'delay_rate'
conditions  = [ data_ML[col] <=0.15, (data_ML[col] >0.15) & (data_ML[col]<= 0.20), 
               (data_ML[col] >0.20) & (data_ML[col]<= 0.25), data_ML[col]> 0.25]
choices     = [0, 1, 2, 3]

data_ML['class'] = np.select(conditions, choices, default=np.nan)
data_ML['class'] = data_ML['class'].astype(int)

data_ML.head(10)

(255869, 5)


,year,month,carrier,arr_flights,arr_del15,delay_rate,class
0,0.0,0.5,AA,307.0,56.0,0.182410,1
1,0.0,0.5,AA,90.0,27.0,0.300000,3
2,0.0,0.5,AA,752.0,186.0,0.247340,2
3,0.0,0.5,AA,842.0,174.0,0.206651,2
4,0.0,0.5,AA,383.0,55.0,0.143603,0
5,0.0,0.5,AA,89.0,12.0,0.134831,0
6,0.0,0.5,AA,445.0,82.0,0.184270,1
7,0.0,0.5,AA,1266.0,225.0,0.177725,1
8,0.0,0.5,AA,119.0,27.0,0.226891,2
9,0.0,0.5,AA,593.0,101.0,0.170320,1


In [0]:
from sklearn.preprocessing import LabelEncoder


encoder = LabelEncoder()
data_ML['carrier'] = encoder.fit_transform(data_ML['carrier'])

In [9]:
data_ML.head()

,year,month,carrier,arr_flights,arr_del15,delay_rate,class
0,0.0,0.5,1,307.0,56.0,0.182410,1
1,0.0,0.5,1,90.0,27.0,0.300000,3
2,0.0,0.5,1,752.0,186.0,0.247340,2
3,0.0,0.5,1,842.0,174.0,0.206651,2
4,0.0,0.5,1,383.0,55.0,0.143603,0


In [10]:
y = data_ML['class']
X = data_ML.drop('class', axis=1)

X.head()

,year,month,carrier,arr_flights,arr_del15,delay_rate
0,0.0,0.5,1,307.0,56.0,0.182410
1,0.0,0.5,1,90.0,27.0,0.300000
2,0.0,0.5,1,752.0,186.0,0.247340
3,0.0,0.5,1,842.0,174.0,0.206651
4,0.0,0.5,1,383.0,55.0,0.143603


In [11]:
print(X.shape)
print(y.shape)

(255869, 6)
(255869,)


In [0]:
X = X.as_matrix()
y = y.as_matrix()

In [13]:
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(204695, 6)
(51174, 6)
(204695,)
(51174,)


In [14]:
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Using TensorFlow backend.


In [15]:
print(y_train.shape)
print(y_test.shape)

(204695, 4)
(51174, 4)


In [0]:
from keras import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(40000,activation='relu',kernel_initializer ='random_normal', input_dim=6))
model.add(Dense(400,activation='relu',kernel_initializer ='random_normal'))
model.add(Dense(4,activation='softmax',kernel_initializer = 'random_normal'))

In [18]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 40000)             280000    
_________________________________________________________________
dense_5 (Dense)              (None, 400)               16000400  
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 1604      
Total params: 16,282,004
Trainable params: 16,282,004
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train, batch_size=30, epochs=10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





204695/204695 [==============================] - 110s 539us/step - loss: 11.4164 - acc: 0.2917
Epoch 2/10
204695/204695 [==============================] - 100s 488us/step - loss: 11.4133 - acc: 0.2919
Epoch 3/10
204695/204695 [==============================] - 99s 486us/step - loss: 11.4133 - acc: 0.2919
Epoch 4/10
204695/204695 [==============================] - 99s 484us/step - loss: 11.4133 - acc: 0.2919
Epoch 5/10
204695/204695 [==============================] - 99s 485us/step - loss: 11.4133 - acc: 0.2919
Epoch 6/10
204695/204695 [==============================] - 99s 484us/step - loss: 11.4133 - acc: 0.2919
Epoch 7/10
204695/204695 [==============================] - 99s 483us/step - loss: 11.4133 - acc: 0.2919
Epoch 8/10
204695/204695 [==============================] - 99s 483us/step - loss: 11.4133 - acc: 0.2919
Epoch 9/10
204695/204695 [==============================

In [0]:
y_pred = model.predict(X_test)

In [25]:
from sklearn.metrics import confusion_matrix,classification_report

print(classification_report(y_test,y_pred))

ValueError: ignored

In [34]:
y_pred[y_pred == [0,0,1,0]].sum()

0.0

In [36]:
y_train

array([[0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.]], dtype=float32)